In [4]:
import sys
sys.path.append('../Codes/library')
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from Immuno_models import*
from io import StringIO
from matplotlib.lines import Line2D
from datetime import datetime, timedelta
import scipy.special as sc
import os.path
import pickle
from matplotlib import style
from scipy.interpolate import interp1d
from scipy.optimize import curve_fit
from IPython.display import display, Math
import json

Text_files_path = '/Users/robertomorantovar/Dropbox/Research/Evolution_Immune_System/Text_files/'


N_A = 6.02214076e23
k_BT = 1.380649e-23*293
style.use('seaborn-paper')
print(style.available)
%autosave 60

['Solarize_Light2', '_classic_test_patch', 'bmh', 'classic', 'dark_background', 'fast', 'fivethirtyeight', 'ggplot', 'grayscale', 'seaborn', 'seaborn-bright', 'seaborn-colorblind', 'seaborn-dark', 'seaborn-dark-palette', 'seaborn-darkgrid', 'seaborn-deep', 'seaborn-muted', 'seaborn-notebook', 'seaborn-paper', 'seaborn-pastel', 'seaborn-poster', 'seaborn-talk', 'seaborn-ticks', 'seaborn-white', 'seaborn-whitegrid', 'tableau-colorblind10']


Autosaving every 60 seconds


In [38]:
def binding_affinity(x, a, b):
    return 1/(1+((np.exp(a+b*x))))

In [ ]:
a = -6
b = 2
N_epitopes = np.arange(1, 12, 1)
rho = 1
tau = 2
T = 4000*tau
R_bar = np.array([])
for g in N_epitopes:
    n_mutations = np.zeros(g) #Array with number of mutatios in each epitope
    neutralizations = binding_affinity(n_mutations, a, b) #Array with individuals neutralization probabilities
    time = np.array([0]) #Array with time
    t = time[-1]
    time_intervals = np.array([])
    exposure_times = np.array([0])
    sick_times = np.array([])
    R = np.array([1-np.product(1-neutralizations)]) #Array with total recognition function
    propensities = np.array(np.concatenate(([rho]*g, [1/tau]))) #Array with propensities
    i = 0
    while (t<T):
        i+=1
        cumsum = np.cumsum(propensities)
        alpha = np.sum(propensities)
        r1 = np.random.rand()
        dti = (1/alpha)*np.log(float(1/r1))
        time_intervals = np.append(time_intervals, dti)
        t = t+dti
        time = np.append(time, t)
        r2 = np.random.rand()
        idEvent = np.searchsorted(cumsum,r2*alpha)
        if(idEvent<g):#mutation
            n_mutations[idEvent]+=1
        else:#exposure
            exposure_times = np.append(exposure_times, t)
            r_sick = np.random.rand()
            if(r_sick>R[-1]): #sick
                n_mutations = np.zeros(g) #update mutations
                sick_times = np.append(sick_times, t)
        neutralizations = binding_affinity(n_mutations, a, b) #update neutralizations  
        R = np.append(R, 1-np.product(1-neutralizations)) #update R
    
    #R_bar = np.append(R_bar, np.sum(R[:-1]*time_intervals)/time[-1])
    R_bar = np.append(R_bar, 1-np.size(sick_times)/np.size(exposure_times))


In [ ]:
plt.plot(N_epitopes, 1-R_bar, 'o')

In [ ]:
plt.hist(exposure_times[2:]-exposure_times[1:-1], bins = np.arange(0, 35))
np.mean(exposure_times[2:]-exposure_times[1:-1])

In [ ]:
fig, ax = plt.subplots(figsize = (20,8))
ax.step(time, R, '.-')
ax.plot(sick_times, np.zeros_like(sick_times), 'o')
ax.vlines(exposure_times, 0, 1, linestyle = '--', color = 'grey')
ax.hlines(np.sum(R[:-1]*time_intervals)/np.sum(time_intervals), 0, T, linestyle = '--')
ax.set_xlim(0, 30)
#ax.hlines(np.mean(R), 0, T, linestyle = 'dotted')

In [319]:
print(np.sum(time_intervals), T, time[-1])

30000.253536064032 30000 30000.253536064647
